In [8]:
import re
from fastai.text import * 

import html

In [9]:
chunksize=24000

BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('data/aclImdb/')

CLAS_PATH=Path('data/imdb_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('data/imdb_lm/')
LM_PATH.mkdir(exist_ok=True)

In [4]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [34]:
def get_texts(df, n_lbls=1):
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts  = '\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    
    for i in range(n_lbls+1, len(df.columns)): 
        texts += ' {FLD} {i-n_lbls} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)

    tok = Tokenizer().proc_all_mp(partition_by_cores(texts))
    return tok, list(labels)

In [17]:
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [29]:
df_trn = pd.read_csv(LM_PATH/'train.csv', header=None)#, chunksize=chunksize)
df_val = pd.read_csv(LM_PATH/'test.csv', header=None)#, chunksize=chunksize)

In [30]:
df_trn.head()

,0,1
0,0,I bought the video because the plot sounded li...
1,0,"This movie is so bad, I knew how it ends right..."
2,0,I saw this film with a special screening of th...
3,0,I was going to give this a 4 but it gets a 10....
4,0,Based on a Phillip K. Dick story (the writer o...


In [35]:
tok_trn, trn_labels = get_all(df_trn, 1)
tok_val, val_labels = get_all(df_val, 1)

0
0


In [36]:
(LM_PATH/'tmp').mkdir(exist_ok=True)

In [37]:
np.save(LM_PATH/'tmp'/'tok_trn.npy', tok_trn)
np.save(LM_PATH/'tmp'/'tok_val.npy', tok_val)

In [38]:
tok_trn = np.load(LM_PATH/'tmp'/'tok_trn.npy')
tok_val = np.load(LM_PATH/'tmp'/'tok_val.npy')

In [39]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

[('the', 1012663),
 (',', 827666),
 ('.', 825186),
 ('and', 493048),
 ('a', 489065),
 ('of', 439682),
 ('to', 407014),
 ('is', 330850),
 ('it', 285903),
 ('in', 282471),
 ('t_up', 255616),
 ('i', 255503),
 ('this', 225939),
 ('that', 219592),
 ('"', 198354),
 ("'s", 186208),
 ('{', 182892),
 ('}', 182868),
 ('-', 157980),
 ('was', 150893),
 ('\n\n', 150013),
 ('as', 139703),
 ('with', 133756),
 ('for', 133463),
 ('movie', 131608)]

In [40]:
max_vocab = 60000
min_freq = 2

In [41]:
itos = [o for o,c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [42]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

60002

In [43]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [45]:
np.save(LM_PATH/'tmp'/'trn_ids.npy', trn_lm)
np.save(LM_PATH/'tmp'/'val_ids.npy', val_lm)

In [47]:
pickle.dump(itos, open(str(LM_PATH/'tmp'/'itos.pkl'), 'wb'))

In [49]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids.npy')
val_lm = np.load(LM_PATH/'tmp'/'val_ids.npy')
itos = pickle.load(open(str(LM_PATH/'tmp'/'itos.pkl'), 'rb'))

In [50]:
vs=len(itos)
vs,len(trn_lm)

(60002, 75000)

## Wikitext103

In [52]:
!wget -nH -r -np -P {PATH} http://files.fast.ai/models/wt103/fwd_wt103.h5

--2019-01-13 23:13:48--  http://files.fast.ai/models/wt103/fwd_wt103.h5
Resolviendo files.fast.ai (files.fast.ai)... 67.205.15.147
Conectando con files.fast.ai (files.fast.ai)[67.205.15.147]:80... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 462387687 (441M) [text/plain]
Grabando a: “data/aclImdb/models/wt103/fwd_wt103.h5”

models/wt103/fwd_wt 100%[===================>] 440,97M  1,19MB/s    in 15m 24s 

2019-01-13 23:29:13 (489 KB/s) - “data/aclImdb/models/wt103/fwd_wt103.h5” guardado [462387687/462387687]

ACABADO --2019-01-13 23:29:13--
Tiempo total de reloj: 15m 25s
Descargados: 1 ficheros, 441M en 15m 24s (489 KB/s)


In [53]:
em_sz,nh,nl = 400,1150,3

In [54]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [56]:
wgts = torch.load(str(PRE_LM_PATH), map_location=lambda storage, loc: storage)

In [57]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

In [60]:
itos2 = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [ ]:
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [ ]:
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder_with_dropout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))